In [1]:
!nvidia-smi

Fri Feb  3 09:03:45 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P0    26W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Reshape, Dropout, Dense, Multiply, Dot, Concatenate, Embedding

In [3]:
tf.__version__

'2.9.2'

In [10]:
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
import tensorflow.keras as keras
from tensorflow.keras import models, layers, optimizers
import tensorflow as tf
from keras.utils import np_utils
from keras.models import load_model
from keras.preprocessing import image
from keras.datasets import cifar10
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2

In [5]:
#@title Default title text
conv_base = InceptionV3(weights="imagenet", include_top=False, input_shape=(256, 256, 3))

87910968/87910968 [==============================] - 1s 0us/step


In [6]:
# Summary of pre-trained model
conv_base.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 127, 127, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 127, 127, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                      

In [9]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

X_train = X_train / 255.0
X_test = X_test / 255.0

y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)

X_train.shape, y_train.shape, X_test.shape, y_test.shape 

170498071/170498071 [==============================] - 2s 0us/step


((50000, 32, 32, 3), (50000, 10), (10000, 32, 32, 3), (10000, 10))

In [13]:
model = models.Sequential()
#Upsamling increase the row and the column of the dataset by 2 
model.add(layers.UpSampling2D((2, 2)))
model.add(layers.UpSampling2D((2, 2)))
model.add(layers.UpSampling2D((2, 2)))

model.add(conv_base)
model.add(layers.Flatten())

model.add(layers.Dense(128, activation="relu"))
model.add(layers.Dropout(0.5))
model.add(layers.BatchNormalization())

model.add(layers.Dense(64, activation="relu"))
model.add(layers.Dropout(0.5))
model.add(layers.BatchNormalization())

model.add(layers.Dense(10, activation="softmax"))
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

history = model.fit(X_train, y_train, steps_per_epoch=10, epochs=2, batch_size=64, validation_data=(X_test, y_test))

Epoch 1/2
10/10 [==============================] - 53s 4s/step - loss: 2.7118 - accuracy: 0.1609 - val_loss: 6.0422 - val_accuracy: 0.1026
Epoch 2/2
10/10 [==============================] - 30s 3s/step - loss: 2.4157 - accuracy: 0.2000 - val_loss: 3.2837 - val_accuracy: 0.0902


In [14]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 up_sampling2d_6 (UpSampling  (None, 64, 64, 3)        0         
 2D)                                                             
                                                                 
 up_sampling2d_7 (UpSampling  (None, 128, 128, 3)      0         
 2D)                                                             
                                                                 
 up_sampling2d_8 (UpSampling  (None, 256, 256, 3)      0         
 2D)                                                             
                                                                 
 inception_v3 (Functional)   (None, 6, 6, 2048)        21802784  
                                                                 
 flatten_2 (Flatten)         (None, 73728)             0         
                                                      

In [15]:
model.save("cifar_model.h5")

In [16]:
loaded_model = tf.keras.models.load_model("cifar_model.h5")